# This notebook's code originates from @LiamClarkZA's python_3d_toolbox

In [ ]:
import os
import sys
import glob
import numpy as np
import pandas as pd
import pickle
from scipy.sparse import lil_matrix
from scipy.optimize import least_squares
import cv2
import time

import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
plt.style.use('../configs/mplstyle.yaml')

from lib import calib, app, utils, plotting
from lib.misc import get_markers
from lib.app import reconstruction_reprojection_video
# need to import app before plotting!!!!!!

%load_ext autoreload
%autoreload 2

ROOT_DATA_DIR = os.path.join("..", "data")

# Define the params in the cell below. Thereafter, run all!

In [ ]:
DATA_DIR = os.path.join(ROOT_DATA_DIR, "2017_08_29 - Copy", "top", "jules", "run1_1")

start_frame = 20
end_frame = 70

# DLC p_cutoff - any points with likelihood < dlc_thresh are not trusted in EKF
dlc_thresh = 0.7  # change this only if FTE result is unsatisfactory

# SBA

## Function definitions

In [ ]:
def points_2d_df_to_3d(points_2d_df, k_arr, d_arr, r_arr, t_arr, output_3d_point_df_filepath=None, output_video_filepath=None):
    print("Creating initial 3D estimates for optimisation...\n")
    triangulate_func = calib.triangulate_points_fisheye
    pairwise_points_3d_df = calib.get_pairwise_3d_points_from_df(points_2d_df, k_arr, d_arr, r_arr, t_arr, triangulate_func)
    
    print("Performing sparse bundle adjustment...\n")
    points_3d_df = run_point_bundle_adjustment(pairwise_points_3d_df, points_2d_df, k_arr, d_arr, r_arr, t_arr)
    
    if output_3d_point_df_filepath:
        output_3d_point_df_filepath = os.path.expanduser(output_3d_point_df_filepath)
        print("Saving 3D points...")
        if str(output_3d_point_df_filepath).endswith('.csv'):
            points_3d_df.to_csv(output_3d_point_df_filepath)
        elif str(output_3d_point_df_filepath).endswith('.pickle'):
            points_3d_df.to_pickle(output_3d_point_df_filepath)
        elif str(output_3d_point_df_filepath).endswith('.h5'):
            points_3d_df.to_hdf(output_3d_point_df_filepath)
        else:
            print("Unsupported export format")
        print(f"Done. 3D points saved to {output_3d_point_df_filepath}")
    
    return points_3d_df


def run_point_bundle_adjustment(points_3d_df, points_2d_df, k_arr, d_arr, r_arr, t_arr, cam_model='fisheye'):
    camera_params = []
    for k,d,r,t in zip(k_arr,d_arr,r_arr,t_arr):
        camera_params.extend(get_camera_params(cam_model, k, d, cv2.Rodrigues(r)[0], t))
    n_cameras = len(k_arr)
    
    camera_params = np.array(camera_params)
    points_3d_df['point_index'] = points_3d_df.index
    points_df = points_2d_df.merge(points_3d_df, how='inner', on=['frame','marker'], suffixes=('_cam',''))
    camera_indices = np.array(points_df['camera'], dtype=np.integer)
    point_indices = np.array(points_df['point_index'], dtype=np.integer)
    points_2d = np.array(points_df[['x_cam', 'y_cam']], dtype=np.float)
    points_3d = np.array(points_3d_df[['x', 'y', 'z']], dtype=np.float)
    n_points = len(points_3d)
    if cam_model == 'standard':
        cost_func = standard_point_cost_func
    elif cam_model == 'fisheye':
        cost_func = fisheye_point_cost_func
    else:
        raise InvalidCameraModelError(f"Camera model not supported: {cam_model}")
    
    x0 = points_3d.ravel()
    f0 = cost_func(x0, camera_params, n_cameras, n_points, camera_indices, point_indices, points_2d)
    plt.plot(f0)
    A = create_point_sba_sparsity_matrix(n_points, camera_indices, point_indices)
    t0 = time.time()
    res = least_squares(cost_func, x0, jac_sparsity=A, verbose=2, x_scale='jac', ftol=1e-6, method='trf',
                        args=(camera_params, n_cameras, n_points, camera_indices, point_indices, points_2d), max_nfev=100)
    t1 = time.time()
    print("\nOptimization took {0:.0f} seconds\n".format(t1 - t0))
    plt.plot(f0, label="Cost before")
    plt.plot(res.fun, label="Cost after")
    plt.legend()
    plt.show()
    params = res.x
    new_points_3d = params.reshape((n_points, 3))
    new_points_3d_df = points_3d_df.copy()
    new_points_3d_df['x'] = new_points_3d[:, 0]
    new_points_3d_df['y'] = new_points_3d[:, 1]
    new_points_3d_df['z'] = new_points_3d[:, 2]
    return new_points_3d_df


def get_camera_params(cam_model, K, D, R, T):
    #FISHEYE:  [R, R, R, T, T, T, fx, fy, cx, cy, alpha, k1, k2, k3, k4]
    #STANDARD: [R, R, R, T, T, T, fx, fy, cx, cy, k1, k2, p1, p2, k3]

    params = np.empty(15)
    params[0:3] = R.ravel() # R
    params[3:6] = T.ravel() # T
    params[6] = K[0,0]      # fx
    params[7] = K[1,1]      # fy
    params[8] = K[0,2]      # cx
    params[9] = K[1,2]      # cy
    
    if cam_model == 'fisheye':
        params[10] = K[0,1]       # alpha
        params[11:15] = D.ravel() # k1, k2, k3, k4
    elif cam_model == 'standard':
        params[10:15] = D.ravel() # k1, k2, p1, p2, k3
    
    return params


def fisheye_point_cost_func(params, camera_params, n_cameras, n_points, camera_indices, point_indices, points_2d):
    camera_params = camera_params.reshape((n_cameras, 15))
    points_3d = params.reshape((n_points, 3))
    points_2d_reprojected = project_fisheye(points_3d[point_indices], camera_params[camera_indices])

    error = (points_2d_reprojected - points_2d).ravel()
    return error


def project_fisheye(world_pts, cam_params):
    #camera params: [R, R, R, T, T, T, fx, fy, cx, cy, alpha, k1, k2, k3, k4]
    R = cam_params[:,:3]
    T = cam_params[:,3:6]
    fx = cam_params[:,6]
    fy = cam_params[:,7]
    cx = cam_params[:,8]
    cy = cam_params[:,9]
    alpha = cam_params[:,10]
    k1 = cam_params[:,11]
    k2 = cam_params[:,12]
    k3 = cam_params[:,13]
    k4 = cam_params[:,14]
    #do the reprojection - https://docs.opencv.org/4.1.0/db/d58/group__calib3d__fisheye.html
    pts = rotate(world_pts, R)
    pts += T
    a = pts[:,0]/pts[:,2]
    b = pts[:,1]/pts[:,2]
    r = np.sqrt(a**2 + b**2)
    theta = np.arctan(r)
    theta_d = theta*(1 + k1*theta**2 + k2*theta**4 + k3*theta**6 + k4*theta**8)
    x = theta_d/r*a
    y = theta_d/r*b
    u = fx*(x+alpha*y) + cx
    v = fy*y + cy
    return np.dstack((u,v))


def create_point_sba_sparsity_matrix(n_points, camera_indices, point_indices):
    m = camera_indices.size * 2
    n = n_points * 3
    A = lil_matrix((m, n), dtype=int)
    i = np.arange(camera_indices.size)
    for s in range(3):
        A[2 * i, point_indices * 3 + s] = 1
        A[2 * i + 1, point_indices * 3 + s] = 1
    return A


def rotate(points, rot_vecs):
    # takes numpy arrays of 3D points and corresponding Rodriguez rotation vectors
    # i.e. points[0] = [x,y,z] and camera_params[0] = [R,R,R]
    theta = np.linalg.norm(rot_vecs, axis=1)[:, np.newaxis]
    with np.errstate(invalid='ignore'):
        v = rot_vecs / theta
        v = np.nan_to_num(v)
    dot = np.sum(points * v, axis=1)[:, np.newaxis]
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    return cos_theta * points + sin_theta * np.cross(v, points) + dot * (1 - cos_theta) * v

def save_sba(positions, path, start_frame, dlc_thresh):
    import scipy.io
    types = ["", "_padded"]
    sba_path = os.path.join(path, "sba")
    os.makedirs(sba_path, exist_ok=True)
    
    for t in types:
        
        if "padded" in t:
            nan_arr = np.full((start_frame, positions.shape[1],3), np.nan)
            positions = np.concatenate([nan_arr, positions])
        
        file_data = dict(positions=positions)

        with open(os.path.join(sba_path, f"sba{t}.pickle"), 'wb') as f:
            pickle.dump(file_data, f)
        
        scipy.io.savemat(os.path.join(sba_path, f'sba{t}.mat'), file_data)
        
        if "padded" in t:
            out_fpath = os.path.join(sba_path, "sba.avi")
            reconstruction_reprojection_video(path, out_fpath, positions, include_lure=True, dlc_thresh=dlc_thresh)
        
    print('\nSaved files to', sba_path)

## Perform SBA

In [ ]:
start_frame -= 1

k_arr, d_arr, r_arr, t_arr, cam_res, n_cams, scene_fpath = utils.find_scene_file(DATA_DIR)
d_arr = d_arr.reshape((-1,4))

df_paths = sorted(glob.glob(os.path.join(DATA_DIR, '*.h5')))
points_2d_df = utils.load_dlc_points_as_df(df_paths)
points_2d_df = points_2d_df[points_2d_df['likelihood']>dlc_thresh]

points_3d_df = points_2d_df_to_3d(points_2d_df, k_arr, d_arr, r_arr, t_arr)

## Plot the result!

In [ ]:
#PLOT SBA
markers = get_markers(include_lure=True)

# convert dataframe into 3D joint position vectors for plotting
sba_positions = []
for i in range(start_frame, end_frame):
    frame_i_df  = points_3d_df[points_3d_df['frame']==i]
    frame_i_3d_positions=[]
    
    for marker in markers:
        marker_frame_i_df = frame_i_df[frame_i_df["marker"]==marker]
        marker_frame_i_3d_point = marker_frame_i_df[['x','y','z']].to_numpy()
        # if 3d point vector is empty fill with nans
        if marker_frame_i_3d_point.size == 0:
            marker_frame_i_3d_point = np.empty((1, 3))*np.NaN
            
        frame_i_3d_positions.extend(marker_frame_i_3d_point)
        
    sba_positions.append(np.array(frame_i_3d_positions))

# save_sba(np.array(sba_positions), DATA_DIR, start_frame, dlc_thresh)

with open(os.path.join(DATA_DIR, "sba", 'sba_padded.pickle'), 'rb') as f:
    sba_data = pickle.load(f)
sba_positions = sba_data["positions"]

ca = plotting.Cheetah(sba_positions, DATA_DIR, reprojections=True, dark_mode=True)
ca.animation()